# Projet 1 - MATH2305P
Nom : Edward, Brenton, Simon

## Introductions
- Import les modules nécessaires, `numpy` et `sympy`
- Pour réduire le nombre de variables, nous adoptons cette forme d'équation : $$x+Ay+Bz=C$$

In [10]:
import numpy as np
import sympy as sp

## Construction Basique
- On va constuire les points, droites et plans dans cette partie. Chaque object est représenté de 1, 2 ou 3 équation(s).
- Définir les points, droites, plans comme :
    - `point = Point(((1, 2, 3), (2, 1, 3), (3, 1, 2)))` - 3 paranthèses
    - `droite = Droite(((2, 3, 1), (2, 1, 1)))` - 3 paranthèses
    - `plan = Plan((1, 2, 3))` - 2 paranthèses

- Les paramètres :
    - Équation de l'object : `OBJECT.eqs[INDEX].eq`, INDEX peut être 1, 2 ou 3, dépend de ce que vous choissisez.
    - Paramètre de chaque l'équation : `OBJECT.eqs[INDEX].B_real` pour les valeurs réels, et `OBJECT.eqs[INDEX].B` pour les alphabets.

- Une choses très très très très important : On sais que Python commence par $0$. Donc, dans notre projet, on néglige tous les éléments sur l'index $0$. C'est-à-dire, `eqs[1]` représente la première équation, pas la deuxième équation. `eqs[0]` n'a pas le sens.

In [11]:
x, y, z = sp.symbols("x y z")

"""Representation de chaque équations"""
class Equation():
    global x, y, z
    def __init__(self, nombre=None, args=None):
        if nombre:
            self.A, self.B, self.C = sp.symbols("A[{}] B[{}] C[{}]".format(nombre, nombre, nombre))
        else:
            self.A, self.B, self.C = sp.symbols("A B C")

        # L'équation est exprimée sur la forme : x + Ay + Bz - C = 0
        self.eq =      1 * x + \
                  self.A * y + \
                  self.B * z - \
                  self.C
        
        # Si l'équation a une valeur réelle, substituez-la dans l'équation
        if not args:
            self.real = False
        else:
            self.A_real, self.B_real, self.C_real = args
            self.eq = self.eq.subs({self.A: self.A_real,
                                    self.B: self.B_real,
                                    self.C: self.C_real})

Equation().eq

A*y + B*z - C + x

In [12]:
class Plan(Equation):
    def __init__(self, args=None):
        nombre = 1
        if args:
            self.eqs = [0, Equation(nombre, args)]
        else:
            self.eqs = [0, Equation(nombre)]

            
class Droite(Equation):
    def __init__(self, args=None):
        nombre = 2
        if args:
            self.eqs = [0, Equation(nombre, args[0]), Equation(nombre, args[1])]
        else:
            self.eqs = [0, Equation(nombre), Equation(nombre)]
            
class Point(Equation):
    def __init__(self, args=None):
        nombre = 3
        if args:
            self.eqs =[0, Equation(nombre, args[0]), Equation(nombre, args[1]), Equation(nombre, args[2])]
        else:
            self.eqs = [0, Equation(nombre), Equation(nombre), Equation(nombre)] 
            
Plan((2, 3, 1)).eqs[1].eq

x + 2*y + 3*z - 1

## Opération (a): Parallèlement

> Dans cette partie, on va construire un plan (1 équation) passant par un point (3 équations) et parallèle à un plan donné (1 équation).

- Notations:
    - Le plan donné : $$x+My+Nz=P$$
    - Le point donné par : $$\begin{cases}x+A_1y+B_1z=C_1 \quad(\mathscr{H}_1)\\x+A_2y+B_2z=C_2\quad(\mathscr{H}_2)\\x+A_3y+B_3y=C_3\quad(\mathscr{H}_3)\end{cases}$$Cela implique une point notant $\bar{x}\in \mathbb R^3$ suffit à la fois ces trois équations est : $$\bar{x} \in \mathscr{H}_1\cap \mathscr{H}_2 \cap\mathscr{H}_3$$
    - Les trois hyperplans correspondent : $$\begin{cases}\mathscr{H}_1 = \text{Ker }(\varphi_1),\quad \varphi_1:(x,y,z)\mapsto x+a_1y+B_1z-C_1\\ \mathscr{H}_2 = \text{Ker }(\varphi_2),\quad \varphi_2:(x,y,z) \mapsto x+a_2y+B_2z-C_3\\ \mathscr{H}_3 = \text{Ker }(\varphi_3),\quad \varphi_3:(x,y,z)\mapsto x+a_3y+B_2z-C_3\end{cases}$$
    
- D'après le **Faisceau d'hyperplan**, on sais que si une sous-espace affine $\mathscr{P}=\text{Ker }(p)$ suffit $$\mathscr{P}=\text{Ker }(p) \supset \bigcap_{i=1}^3\mathscr{H}_i = \bigcap_{i=1}^3\text{Ker }\varphi_i$$Alors, $$p = \text{Vect }(\{\varphi_1,\varphi_2, \varphi_3\})$$

- Ça sert à quoi ? Le plan que l'on souhaite contient le point commune de ces hyperplans $\bar{x}$, qui est l'intersection de tous les $(\mathscr{H}_i)$. Donc le plan peut exprimer comme $\mathscr{P}=\text{Ker }(p)$. Cela nous inspire de trouver l'équation étant égale à $0$, la combinaison linéaire de $\{\varphi_1,\varphi_2, \varphi_3\}$.

- En admettant que $(\lambda, \mu, \gamma)\in \mathbb R^3$, On exprime le plan comme $$\lambda(x+A_1y+B_1z-C_1)+ \mu(x+A_2y+B_2z-C_2)+\gamma(x+A_3y+B_3z-C_3)=0\quad(\mathscr{P})$$ Est-ce qu tous les coefficients suffisent ? Évidemment non, en fait le point $\bar{x}$ est aussi sur ce plan, il est à la fois suffit : $$x+My+Nz=P$$.

In [15]:
import sympy as sp
sp.init_printing()
X,Y,Z,A1,A2,A3,B1,B2,B3,C1,C2,C3,M,N,P,M1,M2,N1,N2,P1,P2,u1,u2,u3,L_1,L_2,\
=sp.symbols('X,Y,Z,A1,A2,A3,B1,B2,B3,C1,C2,C3,M,N,P,M1,M2,N1,N2,P1,P2,u1,u2,u3,L_1,L_2')


In [17]:
'''
Opération a 
On sait que le plan passe bien par le point déterminé par trois équations,donc le plan est forcément une combinaision 
linéaire de ces trois équations, alors on peut savoir les coéfficients d'une des combinaision possibles par la fonction 'solve',
si le plan que l'on obtient est parallèle au plan donné.

Le point:
X+A1*Y+B1*Z=C1
X+A2*Y+B2*Z=C2
X+A3*Y+B3*Z=C3

Le plan parallèle
X+M*Y+N*Z=P
'''
Q = sp.solve([u1+u2+u3-1,
              u1*A1+u2*A2+u3*A3-M,
              u1*B1+u2*B2+u3*B3-N],
             [u1,u2,u3])
H = sp.Matrix([[1,A1,B1,-C1],
               [1,A2,B2,-C2],
               [1,A3,B3,-C3]])
Result_1 = sp.Matrix([u1, u2, u3]).subs(Q).transpose()*H
Result_1.simplify()
Result_1

⎡         -C₁⋅(A₂⋅B₃ - A₂⋅N - A₃⋅B₂ + A₃⋅N + B₂⋅M - B₃⋅M) + C₂⋅(A₁⋅B₃ - A₁⋅N -
⎢1  M  N  ────────────────────────────────────────────────────────────────────
⎣                                                           A₁⋅B₂ - A₁⋅B₃ - A₂

 A₃⋅B₁ + A₃⋅N + B₁⋅M - B₃⋅M) - C₃⋅(A₁⋅B₂ - A₁⋅N - A₂⋅B₁ + A₂⋅N + B₁⋅M - B₂⋅M)⎤
─────────────────────────────────────────────────────────────────────────────⎥
⋅B₁ + A₂⋅B₃ + A₃⋅B₁ - A₃⋅B₂                                                  ⎦